# Préparation données & librairies

In [2]:
import numpy as np
import pandas as pd

from data.DataManager import DataManager
from main.CrossRandomnessAnalysis import CrossRandomnessAnalysis
from data.DataCollectors import HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies, plot_3D
from utils.BlockConstructors import cross_overlapping_blocks, cross_non_overlapping_blocks 
from utils.MultiTester import CrossMultiTester


In [3]:
'''
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
'''

asset_pairs = ["BTCUSDT", "ETHUSDT"]

symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }

aggregation_levels = [1, 5, 10, 25, 50, 75, 100, 200, 500, 1000]
year, month = 2025, 2

In [3]:
historical_collector = HistoricalDataCollectorParquet(pairs=asset_pairs, year=year, month=month)
historical_collector.collect()

[SYSTEM] Processing BTCUSDT...
[SYSTEM] Downloading BTCUSDT-trades-2025-02.zip from https://data.binance.vision/data/spot/monthly/trades/BTCUSDT/BTCUSDT-trades-2025-02.zip ...
[SYSTEM] File downloaded: BTCUSDT-trades-2025-02.zip
[SYSTEM] Extracting data/raw_data\BTCUSDT-trades-2025-02.zip ...
[SYSTEM] Extraction finished.
[SYSTEM] Converting BTCUSDT-trades-2025-02.csv → Parquet ...
[SYSTEM] Saved parquet to: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[SYSTEM] Deleted temporary CSV file: data/raw_data\BTCUSDT-trades-2025-02.csv
[SYSTEM] Deleted ZIP: data/raw_data\BTCUSDT-trades-2025-02.zip
[SYSTEM] Processing ETHUSDT...
[SYSTEM] Downloading ETHUSDT-trades-2025-02.zip from https://data.binance.vision/data/spot/monthly/trades/ETHUSDT/ETHUSDT-trades-2025-02.zip ...
[SYSTEM] File downloaded: ETHUSDT-trades-2025-02.zip
[SYSTEM] Extracting data/raw_data\ETHUSDT-trades-2025-02.zip ...
[SYSTEM] Extraction finished.
[SYSTEM] Converting ETHUSDT-trades-2025-02.csv → Parquet ...
[SYSTEM] 

# Analyse des transactions

In [ ]:
tx_df = DataManager.summarize_transaction_characteristics(
    pairs=asset_pairs,
    symbols=symbols,
    year=year,
    month=month,
    aggregation_levels=aggregation_levels,
    max_interval_sec=1.0
)

tx_df

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


KeyboardInterrupt: 

In [3]:
fig = DataManager.plot_transaction_counts_express(
    pairs=asset_pairs,
    symbols=symbols,
    year=year, month=month, day=None,
    aggregation_levels=aggregation_levels
)
fig.show()

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


MemoryError: Unable to allocate 3.84 GiB for an array with shape (4, 128881324) and data type float64

#  Cross KL Divergence

In [4]:
alpha_eth = 5
# Least liquid pair is ETHUSDT
dm_eth = DataManager(['ETHUSDT'], symbols, year, month, None,
                     aggregation_level=alpha_eth)
n_blocks_eth = len(dm_eth.datasets['ETHUSDT']['symbol'])

# BTC is the most liquid pair, we use it to match the aggregation level of ETH to have the same number of blocks
alpha_btc = dm_eth.matching_aggregation_for("BTCUSDT", blocks=n_blocks_eth, year=year, month=month, day=None)

dm_btc = DataManager(['BTCUSDT'], symbols, year, month, None,
                      aggregation_level=alpha_btc)


[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


## Test simple 

In [6]:
k = 3   

symbols_btc = dm_btc.datasets['BTCUSDT']['symbol']
symbols_eth = dm_eth.datasets['ETHUSDT']['symbol']

cra = CrossRandomnessAnalysis(
    symbols_context=symbols_btc,
    symbols_target =symbols_eth,
    k=k,
    s=2,
    asset_context='BTCUSDT',
    alpha_context=alpha_btc,
    asset_target ='ETHUSDT',
    alpha_target =alpha_eth
)

freq_df    = cra.compute_cross_frequencies()
res_bias   = cra.entropy_bias_test()       # uses non-overlapping blocks 
res_kl     = cra.KL_divergence_test()      # uses overlapping blocks 

s = 2  
n_eth = len(dm_eth.datasets["ETHUSDT"]["symbol"])
k_opt_eth = int(np.floor(0.5 * np.log(n_eth) / np.log(s)))
print(f"Optimal k for BTC: {k_opt_eth}")

Optimal k for BTC: 12


In [ ]:
freq_df.set_index('block', inplace=True)

,block,absolute frequency,relative frequency,asset,alpha,block_size
0,0,14475365,0.415224,BTCUSDT,1,3
1,1,539745,0.015483,BTCUSDT,1,3
2,2,2159760,0.061952,BTCUSDT,1,3
3,3,524768,0.015053,BTCUSDT,1,3
4,4,539745,0.015483,BTCUSDT,1,3
5,5,2144783,0.061523,BTCUSDT,1,3
6,6,524769,0.015053,BTCUSDT,1,3
7,7,13952682,0.400230,BTCUSDT,1,3
8,0,8117376,0.457383,ETHUSDT,5,1
9,1,9630047,0.542617,ETHUSDT,5,1


In [26]:
res_bias

,Entropy Bias test
Bias,8437447.034131
Quantile 90%,22.30713
Quantile 95%,24.99579
Quantile 99%,30.577914
P-value,0.0
Mean,15
Hypothesis 1,True


In [27]:
res_kl

,NP Statistic test
NP Statistic,37.666349
Quantile 90%,12.017037
Quantile 95%,14.06714
Quantile 99%,18.475307
P-value,0.000004
Mean,7
Hypothesis 1,True


## Multi-test

In [31]:
import importlib

import utils.MultiTester
importlib.reload(utils.MultiTester)

from utils.MultiTester import CrossMultiTester


In [32]:
cmt = CrossMultiTester('BTCUSDT', 'ETHUSDT')

df_blocks = cmt.test_by_block_size(
    test='NP Statistic',
    max_block_size=10,
    alpha_context=5,
    year=year, month=month, day=None
)
display(df_blocks)

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-0

,Alpha target,Test statistic,Quantile 90,Quantile 95,Quantile 99,Mean
Block size,,,,,,
1,1,3.012283,2.705543,3.841459,6.634897,1
2,1,4.584474,6.251389,7.814728,11.344867,3
3,1,26.589201,12.017037,14.067140,18.475307,7
4,1,49.738251,22.307130,24.995790,30.577914,15
5,1,75.915255,41.421736,44.985343,52.191395,31
6,1,121.625481,77.745385,82.528727,92.010024,63
7,1,211.370810,147.804813,154.301516,166.987390,127
8,1,376.257517,284.335908,293.247835,310.457388,255
9,1,646.159732,552.373933,564.696133,588.297794,511


In [33]:
df = cmt.test_by_aggregation_level(
    test='NP Statistic',
    aggregation_levels=[1,2,5,10,20,50],
    k=3,
    year=year, month=month, day=None
)
df

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-0

,Alpha target,Test statistic,Quantile 90,Quantile 95,Quantile 99,Mean
Aggregation level,,,,,,
1,1,86.840053,12.017037,14.06714,18.475307,7
2,1,107.584261,12.017037,14.06714,18.475307,7
5,1,26.589201,12.017037,14.06714,18.475307,7
10,2,42.508152,12.017037,14.06714,18.475307,7
20,5,5.595405,12.017037,14.06714,18.475307,7
50,12,23.936752,12.017037,14.06714,18.475307,7


In [ ]:
cmt = CrossMultiTester('BTCUSDT', 'ETHUSDT')

result_3D = cmt.test_grid(
    test='NP Statistic',
    list_aggregations=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25,30,35,40,50],
    list_block_sizes=[1,2,3,4,5,6,7,8,9,10,11,12],
    year=year, month=month, day=None
)


plot_3D(result_3D, test='NP Statistic')

[BTCUSDT→ETHUSDT] Cross test grid:   0%|          | 0/60 [00:00<?, ?it/s]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DEBUG] self.blocks_ctx will be created


[BTCUSDT→ETHUSDT] Cross test grid:   2%|▏         | 1/60 [02:01<1:59:19, 121.34s/it]

[DEBUG] self.blocks_ctx will be created


[BTCUSDT→ETHUSDT] Cross test grid:   3%|▎         | 2/60 [02:23<1:00:52, 62.97s/it] 

...

[BTCUSDT→ETHUSDT] Cross test grid:  97%|█████████▋| 58/60 [22:30<00:09,  4.99s/it]

[DEBUG] self.blocks_ctx will be created


[BTCUSDT→ETHUSDT] Cross test grid:  98%|█████████▊| 59/60 [22:32<00:04,  4.12s/it]

[DEBUG] self.blocks_ctx will be created


[BTCUSDT→ETHUSDT] Cross test grid: 100%|██████████| 60/60 [22:34<00:00, 22.58s/it]


In [36]:
df_grid = CrossMultiTester.to_dataframe_from_grid(
    result_3D,
    aggregation_levels=[1,2,5,10,20,50],
    block_sizes=[1,2,3,4,5,6,7,8,9,10],
    test_name='NP Statistic'
)

df_grid

,Aggregation level,Block size,NP Statistic statistic,Quantile 99
0,1,1,13.066114,6.634897
1,1,2,51.203086,11.344867
2,1,3,86.840053,18.475307
3,1,4,134.741226,30.577914
4,1,5,187.738677,52.191395
5,1,6,261.197461,92.010024
6,1,7,363.950658,166.987390
7,1,8,516.345216,310.457388
8,1,9,797.706539,588.297794
9,1,10,1402.791104,1131.158739
